<a href="https://colab.research.google.com/github/jeremysb1/data_analysis_projects/blob/main/metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Metrics Project

The business has just been through its first winter sale. Stakeholders are interested in knowing which products performed best during the Christmas period so they can streamline the products they advertise in future sale periods.

I will do this by analyzing up to two months of events data, presented as two separate data sources with identical structures. Events refer not just to sales but to more general customer actions, such as viewing a product on the website or placing it into their virtual cart.

Some aspects they care about when ranking their products:

*  Volume of sales;
*  Total revenue from a single product;
*  Popularity measured by the number of unique customers who bought a product;
*  Conversion, meaning the percentage of time a product is bought once it has been placed in the virtual shopping cart; and
*  Products with increased performance from November to December.

The output of the analysis should be recommendations of the best-performing products on whichever dimension(s) I choose.

The recommendation could be for single product IDs or broader ones relating to brands or product categories, as long as your choice of “best” product is justified by my analysis.
`


A Minimum Viable Answer should contain:

- Measurements of our chosen metrics at a product level (e.g., a table of total revenue in December by product, sorted in descending revenue order)
- A ranking so we can highlight top performers
Justifications for which metrics were chosen
- A summary of our findings in tabular or visual form
- Recommendations for further iterations (if requested)



In [1]:
import pandas as pd
import numpy as np

november = pd.read_csv('/content/drive/MyDrive/Data Analysis Projects/Metrics/november.csv')
december = pd.read_csv('/content/drive/MyDrive/Data Analysis Projects/Metrics/december.csv')

events = pd.concat([november, december], axis=0, ignore_index=True)
print(events.shape)

(7033125, 9)
